# R Bootcamp Part 5

## stargazer, xtable, robust standard errors, and fixed effects regressions


This bootcamp will help us get more comfortableusing **stargazer** and **xtable** to produce high-quality results and summary statistics tables, and using `felm()` from the **lfe** package for regressions (both fixed effects and regular OLS).


For today, let's load a few packages and read in a dataset on residential water use for residents in Alameda and Contra Costa Counties. 

## Preamble
Here we'll load in our necessary packages and the data file

In [5]:
library(tidyverse)
library(haven)
#library(lfe)
library(stargazer)
library(xtable)

# load in wateruse data, add in measure of gallons per day "gpd"
waterdata <- read_dta("wateruse.dta") %>%
    mutate(gpd = (unit*748)/num_days)
head(waterdata)

hh,billingcycle,unit,num_days,zip,yearbuilt,homesize,num_baths,num_beds,num_rooms,homeval,lotsize,degree_days,precip,city,gpd
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4,142,61,94506,1991,6055,5.5,5,15,4451858,1.0560147,759.7091,1.60,6,1741.246
1,6,83,62,94506,1991,6055,5.5,5,15,4451858,1.0560147,257.3720,105.86,6,1001.355
9,2,162,62,94563,1978,3400,3.0,4,9,2253082,1.8099862,810.0521,2.48,17,1954.452
9,5,150,61,94563,1978,3400,3.0,4,9,2253082,1.8099862,442.4290,51.93,17,1839.344
11,2,186,61,94507,2004,7453,4.5,3,15,3447285,0.9800046,795.5771,1.42,2,2280.787
11,5,94,58,94507,2004,7453,4.5,3,15,3447285,0.9800046,523.2530,44.66,2,1212.276


# Summary Statistics Tables with xtable

`xtable` is a useful package for producing custom summary statistics tables. let's say we're interested in summarizing water use ($gpd$) and degree days ($degree\_days$) according to whether a lot is less than or greater than one acre ($lotsize_1$) or more than 4 acres ($lotsize_4$):

`homesize <- waterdata %>%`

`    select(hh, billingcycle, gpd, degree_days, lotsize) %>%`

`    drop_na() %>%`

`    mutate(lotsize_1 = ifelse((lotsize < 1), "< 1", ">= 1"),`

`           lotsize_4 = ifelse((lotsize > 4), "> 4", "<= 4"))`

`head(homesize)`


In [6]:
# Delete
homesize <- waterdata %>%

    select(hh, billingcycle, gpd, degree_days, lotsize) %>%

    drop_na() %>%

    mutate(lotsize_1 = ifelse((lotsize < 1), "< 1", ">= 1"),

           lotsize_4 = ifelse((lotsize > 4), "> 4", "<= 4"))

head(homesize)

hh,billingcycle,gpd,degree_days,lotsize,lotsize_1,lotsize_4
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,4,1741.246,759.7091,1.0560147,>= 1,<= 4
1,6,1001.355,257.3720,1.0560147,>= 1,<= 4
9,2,1954.452,810.0521,1.8099862,>= 1,<= 4
9,5,1839.344,442.4290,1.8099862,>= 1,<= 4
11,2,2280.787,795.5771,0.9800046,< 1,<= 4
11,5,1212.276,523.2530,0.9800046,< 1,<= 4


We know how to create summary statistics for these two variables for both levels of $lotsize\_1$ and $lotsize\_4$ using `summarise()`:

`sumstat_1 <- homesize %>%`

`                 group_by(lotsize_1) %>%`

`                  summarise(mean_gpd = mean(gpd), `

`                           mean_degdays = mean(degree_days))`

`sumstat_1`


`sumstat_4 <- homesize %>%`

`                  group_by(lotsize_4) %>%`

`                   summarise(mean_gpd = mean(gpd), `

`                            mean_degdays = mean(degree_days))`

`sumstat_4`

In [9]:
# Delete
sumstat_1 <- homesize %>%

                 group_by(lotsize_1) %>%

                 summarise(mean_gpd = mean(gpd),

                           mean_degdays = mean(degree_days))

sumstat_1

sumstat_4 <- homesize %>%

                  group_by(lotsize_4) %>%

                  summarise(mean_gpd = mean(gpd),

                            mean_degdays = mean(degree_days))

sumstat_4

lotsize_1,mean_gpd,mean_degdays
<chr>,<dbl>,<dbl>
< 1,1490.199,568.8051
>= 1,1961.472,569.2840


lotsize_4,mean_gpd,mean_degdays
<chr>,<dbl>,<dbl>
<= 4,1589.234,568.5734
> 4,2616.090,575.1284


And now we can use `xtable()` to put them into the same table!

`full <- xtable(cbind(t(sumstat_1), t(sumstat_4)))`

`rownames(full)[1] <- "Lotsize Group"`

`colnames(full) <- c("lotsize_1 = 1", "lotsize_1 = 0", "lotsize_4 = 0", "lotsize_4 =1")`

`full`

In [10]:
# Delete
full <- xtable(cbind(t(sumstat_1), t(sumstat_4)))

rownames(full)[1] <- "Lotsize Group"

colnames(full) <- c("lotsize_1 = 1", "lotsize_1 = 0", "lotsize_4 = 0", "lotsize_4 =1")

full

,lotsize_1 = 1,lotsize_1 = 0,lotsize_4 = 0,lotsize_4 =1
,<chr>,<chr>,<chr>,<chr>
Lotsize Group,< 1,>= 1,<= 4,> 4
mean_gpd,1490.199,1961.472,1589.234,2616.090
mean_degdays,568.8051,569.2840,568.5734,575.1284


We now have a table `full` that is an xtable object. 

We can also spit this table out in html or latex form if needed using the `print.xtable()` function on our xtable `full`, specifying `type = "html":

`print.xtable(full, type = "html")`

In [12]:
# Delete
print.xtable(full, type = "html")

<!-- html table generated in R 4.3.2 by xtable 1.8-4 package -->
<!-- Thu Mar 21 09:49:52 2024 -->
<table border=1>
<tr> <th>  </th> <th> lotsize_1 = 1 </th> <th> lotsize_1 = 0 </th> <th> lotsize_4 = 0 </th> <th> lotsize_4 =1 </th>  </tr>
  <tr> <td align="right"> Lotsize Group </td> <td> &lt; 1 </td> <td> &gt;= 1 </td> <td> &lt;= 4 </td> <td> &gt; 4 </td> </tr>
  <tr> <td align="right"> mean_gpd </td> <td> 1490.199 </td> <td> 1961.472 </td> <td> 1589.234 </td> <td> 2616.090 </td> </tr>
  <tr> <td align="right"> mean_degdays </td> <td> 568.8051 </td> <td> 569.2840 </td> <td> 568.5734 </td> <td> 575.1284 </td> </tr>
   </table>


Copy and paste the html code here (or into any other **markdown** cell) to see how it appears.

<!-- html table generated in R 4.3.2 by xtable 1.8-4 package -->
<!-- Thu Mar 21 09:49:52 2024 -->
<table border=1>
<tr> <th>  </th> <th> lotsize_1 = 1 </th> <th> lotsize_1 = 0 </th> <th> lotsize_4 = 0 </th> <th> lotsize_4 =1 </th>  </tr>
  <tr> <td align="right"> Lotsize Group </td> <td> &lt; 1 </td> <td> &gt;= 1 </td> <td> &lt;= 4 </td> <td> &gt; 4 </td> </tr>
  <tr> <td align="right"> mean_gpd </td> <td> 1490.199 </td> <td> 1961.472 </td> <td> 1589.234 </td> <td> 2616.090 </td> </tr>
  <tr> <td align="right"> mean_degdays </td> <td> 568.8051 </td> <td> 569.2840 </td> <td> 568.5734 </td> <td> 575.1284 </td> </tr>
   </table>

# Regression Tables in Stargazer

`stargazer` is a super useful package for producing professional-quality regression tables. While it defaults to producing LaTeX format tables (a typesetting language a lot of economists use), for use in our class we can also produce html code that can easily be copied into text cells and formatted perfectly.

If we run the following three regressions: 

\begin{align*} GPD_{it} &= \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~(1)\\
 GPD_{it} &= \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} + \beta_3 lotsize_{i}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~(2)\\
GPD_{it} &= \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} + \beta_3 lotsize_{i} + \beta_4 Homeval_i~~~~~~~~~~~~~~~~~~(3)
\end{align*}

We might want to present the results side by side in the same table so that we can easily compare coefficients from one column to the other. To do that with `stargazer`, we can 

1. Run each regression, storing them in memory
2. Run `stargazer(reg1, reg2, reg3, ..., type )` where the first arguments are all the regression objects we want in the table, and telling R what type of output we want

If we specify `type = "text"`, we'll get the table displayed directly in the output window:

`reg_a <- lm(gpd ~ degree_days + precip, waterdata)`

`reg_b <- lm(gpd ~ degree_days + precip + lotsize, waterdata)`

`reg_c <- lm(gpd ~ degree_days + precip + lotsize + homeval, waterdata)`

`stargazer(reg_a, reg_b, reg_c, type = "text")`

In [30]:
install.packages("stargazer")
library(stargazer)

# Delete
reg_a <- lm(gpd ~ degree_days + precip, waterdata)

reg_b <- lm(gpd ~ degree_days + precip + lotsize, waterdata)

reg_c <- lm(gpd ~ degree_days + precip + lotsize + homeval, waterdata)

stargazer(reg_a, reg_b, reg_c, type = "text")

Installing package into ‘/srv/r’
(as ‘lib’ is unspecified)



ERROR: Error in (.format.s.statistics.list != "p25") && (.format.s.statistics.list != : 'length = 7' in coercion to 'logical(1)'


And if we specify `type = "html"`, we'll get html code that we need to copy and paste into a text/markdown cell:

`stargazer(reg_a, reg_b, reg_c, type = "html")`

In [28]:
stargazer(reg_a, reg_b, reg_c, type = "html")

ERROR: Error in (.format.s.statistics.list != "p25") && (.format.s.statistics.list != : 'length = 7' in coercion to 'logical(1)'


Now all we need to do is copy and paste that html code from the output into a text cell and we've got our table!

(copy your code here)

And we get a nice looking regression table with all three models side by side! This makes it easy to see how the coefficient on lot size falls when we add in home value, letting us quickly figure out the sign of correlation between the two.

## Table Options

Stargazer has a ton of different options for customizing the look of our table with optional arguments, including
* `title` lets us add a custom title
* `column.labels` lets you add text labels to the columns
* `covariate.labels` lets us specify custom labels for all our variables other than the variable names. Specify each label in quotations in the form of a vector with `c()`
* `ci = TRUE` adds in confidence intervals (by default for the 10\% level, but you can change it to the 1\% level with `ci.level = 0.99`
* `intercept.bottom = FALSE` will move the constant to the top of the table
* `digits` lets you choose the number of decimal places to display
* `notes` lets you add some notes at the bottom

For example, we could customize the above table as

`stargazer(reg_a, reg_b, reg_c, type = "text",
         title = "Water Use, Weather, and Home Characteristics",
         column.labels = c("Weather", "With Lotsize", "With HomeVal"),
          covariate.labels = c("Intercept", "Degree Days", "Precipitation (mm)", "Lot Size (Acres)", "Home Value (USD)"),
          intercept.bottom = FALSE,
          digits = 2,
          note = "Isn't stargazer neat?"
          )`

# Summary Statistics Tables in Stargazer

Can we use Stargazer for summary statistics tables too? You bet we can! 

Stargazer especially comes in handy if we have a lot of variables we want to summarize and one or no variables we want to group them on. This approach works especially well with `across()` within `summarise()`.

For example, let's say we wanted to summarise the median and variance of `gpd`, `precip`, and `degree_days` by whether the home was built after 1980 or not. Rather than create separate tables for all of the variables and merge them together like with xtable, we can just summarise across with

`ss_acr <- mutate(waterdata, pre_80 = ifelse(yearbuilt < 1980, "1. Pre-1980", "2. 1980+")) %>%
    group_by(pre_80) %>%
    summarise(across(.cols = c(gpd, precip, degree_days),
                     .fns = list(Median = median, Variance = var)))
ss_acr`

Note that `ifelse()` is a function that follows the format

`ifelse(Condition, Value if True, Value if False)`

Here our condition is that the $yearbuilt$ variable is less than 1980. If it’s true, we want this
new variable to take on the label "1. Pre-1980", and otherwise
be "2. 1980+".

This table then contains everything we want, but having it displayed "wide" like this is a bit tough to see. If we wanted to display it "long" where there is one column for each or pre-1980 and post-1980 homes, we can just use the transpose function `t()`. Placing that within the `stargazer()` call and specifying that we want html code then gets us

`stargazer(t(ss_acr), type = "html")`

(copy your html code here)

## Heteroskedasticity-Robust Standard Errors 


There are often times where you want to use heteroskedasticity-robust standard errors in place of the normal kind to account for situations where we might be worried about violating our homoskedasticity assumption. To add robust standard errors to our table, we'll take advantage of the `lmtest` and `sandwich` packages. (We have not yet loaded these packages in the preamble. So, if you run below code as is, you obtain an error message that reads "could not find function "coeftest," SO, add the above two packages to your preablem.)

If we want to see the coefficient table from Regression B with robust standard errors, we can use the `coeftest()` function as specified below:

`coeftest(reg_b, vcov = vcovHC(reg_b, type = "HC1"))`

What the `vcovHC(reg_a, type = "HC1")` part is doing is telling R we want to calculate standard errors using the heteroskedasticity-robust approach (i.e. telling it a specific form of the variance-covariance matrix between our residuals). `coeftest()` then prints the nice output table. 

While this is a nice way to view the robust standard errors in a summary-style table, sometimes we want to extract the robust standard errors so we can use them elsewhere - like in stargazer!

To get a vector of robust standard errors from Regression B, we can use the following:

`robust_b <- sqrt(diag(vcovHC(reg_b, type = "HC1")))`

`robust_b`

Which matches the robust standard errors using `coeftest()` earlier. But woah there, that's a function nested in a function nested in *another function*! Let's break this down step-by-step:

`vcov_b <- vcovHC(reg_b, type = "HC1")`

This first `vcov_b` object is getting the entire variance-covariance matrix for our regression coefficients. Since we again specified `type = "HC1"`, we ensure we get the heteroskedasticity-robust version of this matrix (if we had instead specified `type = "constant"` we would be assuming homoskedasticity and would get our usual variance estimates).

What this looks like is


$$VCOV_b = \begin{matrix}{}
\widehat{Var}(\hat \beta_0)       & \widehat{Cov}(\hat \beta_0, \hat\beta_1)  & \widehat{Cov}(\hat \beta_0, \hat\beta_2)  \\
 \widehat{Cov}(\hat \beta_1, \hat\beta_0) & \widehat{Var}(\hat \beta_1)         & \widehat{Cov}(\hat \beta_1, \hat\beta_2) \\
 \widehat{Cov}(\hat \beta_2, \hat\beta_0)  & \widehat{Cov}(\hat \beta_2, \hat\beta_1) & \widehat{Var}(\hat \beta_2)  
\end{matrix}$$

Where each element is $\hat\sigma_i$ in the ith row mutiplied by $\hat\sigma_j$ in the jth column. Note that when $i = j$ in the main diagonal, we get the variance estimate for $\hat \beta_i$!

You can check this by running the following lines:

`vcov_b <- vcovHC(reg_b, type = "HC1")
 vcov_b`

`var_b <- diag(vcov_b)`

The `diag()` function extracts this main diagonal, giving us a vector of our robust estimated variances

`robust_b <- sqrt(var_b)`

And taking the square root gets us our standard error estimates for our $\hat\beta$'s!

See the process by running the following lines:

`var_b <- diag(vcov_b)
 var_b`

`robust_b <- sqrt(var_b)
 robust_b`

## Stargazer and Heteroskedasticity-Robust Standard Errors 

Now that we know how to get our robust standard errors, we can grab them for all three of our regressions and add them to our beautiful stargazer table:

`robust_a <- sqrt(diag(vcovHC(reg_a, type = "HC1")))
robust_b <- sqrt(diag(vcovHC(reg_b, type = "HC1")))
robust_c <- sqrt(diag(vcovHC(reg_c, type = "HC1")))`


`stargazer(reg_a, reg_b, reg_c, 
           type = "html",
          se = list(robust_a, robust_b, robust_c),
          omit.stat = "f")`
          
Here we're adding the robust standard errors to `stargazer()` with the `se =` argument (combining them together in the right order as a list). I'm also omitting the overall F test at the bottom with `omit.stat = "f"` since we'd need to correct that too for heteroskedasticity. 

Try running this code below to see how the standard errors change when we use robust standard errors:

Copy and paste the table code here and run the cell to see it formatted.

Now that looks pretty good, though note that the less than signs in the note for significance labels don't appear right. This is because html is reading the < symbol as a piece of code and not the math symbol. To get around this, you can add dollar signs around the < signs in the html code for the note to have the signs display properly:

`<sup>*</sup>p $<$ 0.1; <sup>**</sup>p $<$ 0.05; <sup>***</sup>p $<$ 0.01</td>`


# Fixed Effects Regression


Today we will practice with fixed effects regressions in __R__. We have two different ways to estimate the model, and we will see how to do both and the situations in which we might favor one versus the other.

Let's give this a try using the dataset `wateruse.dta`. The subset of households are high water users, people who used over 1,000 gallons per billing cycle. We have information on their water use, weather during the period, as well as information on the city and zipcode of where the home is located, and information on the size and value of the house.

Suppose we are interested in running the following panel regression of residential water use:

$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} ~~~~~~~~~~~~~~~~~~~~~~~(1)$$

Where $GPD$ is the gallons used per day by household $i$ in billing cycle $t$, $degree\_days$ the count of degree days experienced by the household in that billing cycle (degree days are a measure of cumulative time spent above a certain temperature threshold), and $precip$ the amount of precipitation in millimeters.

`reg1 <- lm(gpd ~ degree_days + precip, data = waterdata)
summary(reg1)`

Here we obtain an estimate of $\hat\beta_1 = 0.777$, telling us that an additional degree day per billing cycle is associated with an additional $0.7769$ gallon used per day. These billing cycles are roughly two months long, so this suggests an increase of roughly 47 gallons per billing cycle. Our estimate is statistically significant at all conventional levels, suggesting residential water use does respond to increased exposure to high heat.

We estimate a statistically insignificant coefficient on additional precipitation, which tells us that on average household water use in our sample doesn't adjust to how much it rains.

We might think that characteristics of the home impact how much water is used there, so we add in some home controls:


$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it}  + \beta_3 lotsize_{i} + \beta_4 homesize_i + \beta_5 num\_baths_i + \beta_6 num\_beds_i + \beta_7 homeval_i~~~~~~~~~~~~~~~~~~~~~~~(2)$$

`reg2 <- lm(gpd ~ degree_days + precip + lotsize + homesize + num_baths + num_beds + homeval, data = waterdata)
summary(reg2)`

Our coefficient on $degree\_days$ remains statistically significant and doesn't change much, so we find that $\hat\beta_1$ is robust to the addition of home characteristics. Of these characteristics, we obtain statistically significant coefficients on the size of the lot in acres ($lotsize$), the size of the home in square feet ($homesize$), and the number of bedrooms in the home ($num_beds$).

We get a curious result for $\hat\beta_6$: for each additional bedroom in the home we predict that water use will *fall* by 48 gallons per day. 

### Discussion: what might be driving this effect? 

Since there are likely a number of sources of omitted variable bias in the previous model, we think it might be worth including some fixed effects in our model. These will allow us to control for some of the unobserved sources of OVB without having to measure them directly!

## Method 1: Fixed Effects with lm() 

Up to this point we have been running our regressions using the `lm()` function. We can still use `lm()` for our fixed effects models, but it takes some more work and gets increasingly time-intensive as datasets get large.

Recall that we can write our general panel fixed effects model as 

$$ y_{it} = \beta x_{it} + \mathbf{a}_i + \mathbf{d}_t + u_{it} $$

* $y$ our outcome of interest, which varies in both the time and cross-sectional dimensions
* $x_{it}$ our set of time-varying unit characteristics
* $\mathbf{a}_i$ our set of unit fixed effects
* $\mathbf{d}_t$ our time fixed effects

We can estimate this model in `lm()` provided we have variables in our dataframe that correspond to each level of $a_i$ and $d_t$. This means we'll have to generate them before we can run any regression.

### Generating Dummy Variables

In order to include fixed effects for our regression, we can first generate the set of dummy variables that we want. For example, if we want to include a set of city fixed effects in our model, we need to generate them.

We can do this in a few ways.

1. First, we can use `mutate()` and add a separate column for each individual city:

`fe_1 <- waterdata %>%
    mutate(city_1 = as.numeric((city==1)),
           city_2 = as.numeric((city ==2)),
           city_3 = as.numeric((city ==3))) %>%
    select(hh, city, city_1, city_2, city_3)
head(fe_1)`

This can be super tedious though when we have a bunch of different levels of our variable that we want to make fixed effects for. In this case, we have 27 different cities.

2. Alternatively, we can use the `spread()` function to help us out. Here we add in a constant variable `v` that is equal to one in all rows, and a copy of city that adds "city_" to the front of the city number. Then we pass the data to `spread`, telling it to split the variable `cty` into dummy variables for all its levels, with all the "false" cases filled with zeros.

`fe_2 <- waterdata %>%
select(hh, city, billingycle)`


`fe_2 %>%
   mutate(v = 1, cty = paste0("city_", city)) %>% 
   spread(cty, v, fill = 0)`

That is much easier! 

This is a useful approach if you want to produce summary statistics for the fixed effects (i.e. what share of the sample lives in each city), but isn't truly necessary.

Alternatively, we can tell R to read our fixed effects variables as factors:

`lm(gpd ~ degree_days + precip + factor(city), data = waterdata)`

`factor()` around $city$ tells R to split city into dummy variables for each unique value it takes. R will then drop the first level when we run the regression - in our case making the first city our omitted group.


`reg3 <- lm(gpd ~ degree_days + precip + factor(city), data = waterdata)
summary(reg3)`

Now we have everything we need to run the regression


$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it}  + \mathbf{a}_i + \mathbf{d}_t~~~~~~~~~~~~~~~~~~~~~~~(2)$$

Where $\mathbf{a}_i$ are our city fixed effects, and $\mathbf{d}_t$ our billing cycle fixed effects:

`fe_reg1 <- lm(gpd ~ degree_days + precip + factor(city) + factor(billingcycle), data = waterdata)
summary(fe_reg1)`

__R__ automatically chose the first dummy variable for each set of fixed effect (city 1 and billing cycle 1) to leave out as our omitted group. 

Now that we account for which billing cycle we're in (i.e. whether we're in the winter or whether we're in the summer), we find that the coefficient on $degree\_days$ is now much smaller and statistically insignificant. This makes sense, as we were falsely attributing the extra water use that comes from seasonality to temperature on its own. Now that we control for the season we're in via billing cycle fixed effects, we find that deviations in temperature exposure during a billing cycle don't result in dramatically higher water use within the sample.

### Discussion: Why did we drop the home characteristics from our model?

## Method 2: Fixed Effects with felm()

Alternatively, we could do everything way faster using the `felm()` function from the package __lfe__. This package doesn't require us to produce all the dummy variables by hand. Further, it performs the background math way faster so will be much quicker to estimate models using large datasets and many variables.

The syntax we use is now 

`felm(y ~ x1 + x2 + ... + xk | FE_1 + FE_2 + ..., data = df)`

* The first section $y \sim x1 + x2 +... xk$ is our formula, written the same way as with `lm()` - but omitting the fixed effects
* We now add a `|` and in the second section we specify our fixed effects. Here we say $FE\_1 + FE\_2$ which tells __R__ to include fixed effects for each level of the variables $FE\_1$ and $FE\_2$.
* we add the data source after the comma, as before.

Let's go ahead and try this now with our water data model:

`fe_reg2 <- felm(gpd ~ degree_days + precip | city + billingcycle, data = waterdata)
summary(fe_reg2)`

And we estimate the exact same coefficients on $degree\_days$ and $precip$ as in the case where we specified everything by hand! We didn't have to mutate our data or add any variables. The one potential downside is that this approach doesn't report the fixed effects themselves by default. The tradeoff is that `felm` runs a lot faster than `lm`, especially with large datasets. 

We can also recover the fixed effects with getfe(): 

`getfe(fe_reg2, se = TRUE, robust = TRUE)`

the argument `se = TRUE` tells it to produce standard errors too, and `robust = TRUE` further indicates that we want heteroskedasticity-robust standard errors.


Note that this approach doesn't give you the same reference groups as before, but we get the same relative values. Note that before the coefficient on $city2$ was 301.7 and now is 73.9. But the coefficient on $city1$ is -227.8, and if we subtract $city1$ from $city2$ to get the difference in averages for city 2 relative to city 1 we get $73.9 - (-227.8) = 301.7$, the same as before!

# Fixed Effects Practice Question #1

#### From a random sample of agricultural yields Y (1000 dollars per acre) for region $i$ in year $t$ for the US, we have estimated the following eqation:

 \begin{align*} \widehat{\log(Y)}_{it} &=	0.49	+ .01 GE_{it} ~~~~ 	R^2 = .32\\
	&~~~~~(.11)	 ~~~~ (.01)                ~~~~  n = 1526       \end{align*}

#### (a) Interpret the results on the Genetically engineered ($GE$) technology on yields. (follow SSS= Sign Size Significance)

#### (b) Suppose $GE$ is used more on the West Coast, where crop yields are also higher. How would the estimated effect of GE change if we include a West Coast region dummy variable in the equation? Justify your answer.

#### (c) If we include region fixed effects, would they control for the factors in (b)? Justify your answer.

#### (d)  If yields have been generally improving over time and GE adoption was only recently introduced in the USA, what would happen to the coefficient of GE if we included year fixed effects?



# Fixed Effects Practice Question #2

#### A recent paper investigates whether advertisement for Viagra causes increases in birth rates in the USA.  Apparently, advertising for products, including Viagra, happens on TV and reaches households that have a TV within a Marketing region and does not happen in areas outside a designated marketing region. What the authors do is look at hospital birth rates in regions inside and near the advertising region border  and collect data on dollars per 100 people (Ads) for a certain time, and compare those to the birth rates in hospitals located outside and near the advertising region designated border. They conduct a panel data analysis and estimate the following model:

$$ Births_{it} = \beta_0 + \beta_1 Ads + \beta_2 Ads^2 + Z_i + M_t + u_{it}$$

#### Where $Z_i$ are zipcode fixed effects and $M_t$ monthly fixed effects.

#### (a) Why do the authors include Zip Code Fixed Effects? In particular, what would be a variable that they are controlling for when adding Zip Code fixed effects that could cause a problem when interpreting the marginal effect of ad spending on birth rates? What would that (solved) problem be?

#### (b) Why do they add month fixed effects? 

